In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

In [2]:
DATASET_DIR = "dataset"   # inside backend/

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
CLASSES = ["bio-degradable", "non-biodegradable", "other"]


In [3]:
train_gen = ImageDataGenerator(
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

train_data = train_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="training"
)

val_data = train_gen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    subset="validation"
)


Found 2023 images belonging to 3 classes.
Found 504 images belonging to 3 classes.


In [4]:
labels = train_data.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

cw = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", cw)

Class Weights: {0: 0.8450292397660819, 1: 0.6047832585949178, 2: 6.13030303030303}


In [5]:
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # freeze base at first

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dense(3, activation="softmax")  # 3 Classes
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


2025-11-25 15:32:08.875243: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-11-25 15:32:08.875272: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-25 15:32:08.875282: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-25 15:32:08.875305: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-25 15:32:08.875319: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [6]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=cw
)

Epoch 1/20


2025-11-25 15:32:23.903237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


64/64 ━━━━━━━━━━━━━━━━━━━━ 16s 210ms/step - accuracy: 0.7336 - loss: 0.8024 - val_accuracy: 0.9246 - val_loss: 0.3436
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 11s 171ms/step - accuracy: 0.8438 - loss: 0.5220 - val_accuracy: 0.9246 - val_loss: 0.2679
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 11s 173ms/step - accuracy: 0.8700 - loss: 0.4202 - val_accuracy: 0.9067 - val_loss: 0.2664
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 11s 177ms/step - accuracy: 0.8799 - loss: 0.3853 - val_accuracy: 0.9087 - val_loss: 0.2397
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 186ms/step - accuracy: 0.8833 - loss: 0.3431 - val_accuracy: 0.9306 - val_loss: 0.2043
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 186ms/step - accuracy: 0.8927 - loss: 0.3130 - val_accuracy: 0.9206 - val_loss: 0.2248
Epoch 7/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 186ms/step - accuracy: 0.9006 - loss: 0.2924 - val_accuracy: 0.9286 - val_loss: 0.1926
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 188ms/step - accuracy: 0.8972 - loss: 0.2986 - val_accuracy: 0.918

In [7]:
model.export("saved_model")
print("Saved model as SavedModel format.")

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_238')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  5431998800: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  5431998272: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  5409197216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409198272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409197040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409198976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409198448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409198624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409199328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409198800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5409199856: TensorSpec(shape=(), dtype=tf.resou

In [8]:
model.save("waste_model.h5")